### Libraries setup

In [ ]:
#!pip3 install -r requirements.txt
!pip3 install torchinfo numpy pandas nibabel torchio tqdm matplotlib SimpleITK scikit-learn

In [ ]:
#!unzip znorm_cropped_imgs.zip -d /storage

### Model testing

In [2]:
from models.unet3d import UNet
from torchinfo import summary

samp_unet = UNet(1)
summary(samp_unet, (2,1,40,56,72))
#summary(samp_unet, (1,1,196,196,196))

Layer (type:depth-idx)                   Output Shape              Param #
UNet                                     --                        --
├─ModuleList: 1-1                        --                        --
├─ModuleList: 1-2                        --                        --
├─ModuleList: 1-1                        --                        --
│    └─Sequential: 2-1                   [2, 64, 40, 56, 72]       --
│    │    └─Conv3d: 3-1                  [2, 32, 40, 56, 72]       896
│    │    └─BatchNorm3d: 3-2             [2, 32, 40, 56, 72]       64
│    │    └─ReLU: 3-3                    [2, 32, 40, 56, 72]       --
│    │    └─Conv3d: 3-4                  [2, 64, 40, 56, 72]       55,360
│    │    └─BatchNorm3d: 3-5             [2, 64, 40, 56, 72]       128
│    │    └─ReLU: 3-6                    [2, 64, 40, 56, 72]       --
├─MaxPool3d: 1-3                         [2, 64, 20, 28, 36]       --
├─ModuleList: 1-1                        --                        --
│    └─Se

### Model Training

In [ ]:
import torchio as tio

from models.unet3d import UNet
from model_utils.train import train_model, hocv_train_model
from data_utils.dataset import get_train_test_subjects, split_train_subjects

train_subj, test_subj = get_train_test_subjects(
    '/storage/harp_metadata.csv',
    '/storage/znorm_cropped_imgs',
    42,
    True
)

tr_train_subj, tr_val_subj = split_train_subjects('/storage/harp_metadata.csv', train_subj, 18, 42)

firstUNet = UNet(1)
# train_history = train_model(
#     model=firstUNet,
#     dir_name='/storage/znorm_cropped_imgs',
#     brain_side='L',
#     train_ids=train_subj,
#     transforms=[tio.RandomNoise()],
#     num_epochs=100,
#     learning_rate=0.001
# )
train_val_hist = hocv_train_model(
    model = firstUNet,
    dir_name='/storage/znorm_cropped_imgs',
    brain_side='L',
    train_ids=tr_train_subj,
    val_ids=tr_val_subj,
    transforms=[tio.RandomNoise()],
    num_epochs=100,
    learning_rate=0.001
)

In [ ]:
import numpy as np

np.savez('train_hist.npz', **train_history)

In [4]:
train_hist = np.load('train_hist.npz')
train_hist['loss_per_epoch']

array([13.8409085 ,  5.43673003,  3.06899568,  2.03014228,  1.57075645,
        1.24456117,  1.08094809,  0.92620805,  0.84763405,  0.79567849,
        0.73587816,  0.71609001,  0.64892002,  0.63526159,  0.59580059,
        0.57528855,  0.56574512,  0.56601087,  0.53581604,  0.54342163,
        0.52102622,  0.50980311,  0.512585  ,  0.49347595,  0.51221904,
        0.49340566,  0.4920185 ,  0.48519877,  0.4533905 ,  0.48458783,
        0.4825928 ,  0.46858303,  0.46954615,  0.45147895,  0.43158659,
        0.42679915,  0.41532324,  0.42142766,  0.41049762,  0.40914644,
        0.40962956,  0.39471823,  0.37918512,  0.38844658,  0.40158374,
        0.38914997,  0.37942638,  0.37894833,  0.3706524 ,  0.35060388,
        0.34333924,  0.32728564,  0.32884121,  0.32650637,  0.32755369,
        0.32594852,  0.32277594,  0.31097212,  0.30200778,  0.30570154,
        0.29961802,  0.2971015 ,  0.29586184,  0.30350623,  0.28886347,
        0.28516584,  0.28287517,  0.2763304 ,  0.27406437,  0.28